## ●Notebookの内容

前処理&形状変換後データ(コード5の「変数選択後の前処理&形状変換」で作成したもの)の読み込みと左右の入れ込みデータの時点ずらし、統計的性能評価指標と経済的性能評価指標を算出

# 1. 前準備

## 1.1 パッケージのインポート・乱数固定

In [1]:
import pandas as pd #pandasパッケージをインポート
import numpy as np #numpyパッケージをインポート
import torch #ライブラリ「PyTorch」のtorchパッケージをインポート
import torch.nn as nn #「ニューラルネットワーク」モジュールの別名定義
import torch.nn.functional as F #「ニューラルネットワーク・活性化関数」モジュールの別名定義
import collections
import os
import pickle
import optuna
import torch.optim as optim

#乱数の固定
torch.manual_seed(123)

## 1.2 MPSの使用指定

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
print('MPSの使用は',torch.backends.mps.is_available(),'である(Trueなら使用可能、Falseなら使用不可)。')

MPSの使用は True である(Trueなら使用可能、Falseなら使用不可)。


# 2. 整理後データの読み込み

In [3]:
pickle_in = open("./inright_data.pickle","rb") #ファイルの読み込み, 配当込み収益率はランク正規化をしていないデータであることに注意
pickle_in_2 = open("./inleft_data.pickle","rb") #ファイルの読み込み
pickle_in_3 = open("./ranked_data.pickle","rb") #ファイルの読み込み
returns_data_4_0 = pickle.load(pickle_in)
returns_data_4 = returns_data_4_0.iloc[1:,0:].reset_index().iloc[:,1:] #1995/1~2021/12のデータを抽出
data_5_11_0 = pickle.load(pickle_in_2)
data_5_11 = data_5_11_0.iloc[:324,0:].reset_index().iloc[:,1:] #1994/12~2021/11のデータを抽出
data_5_7_0 = pickle.load(pickle_in_3)
data_5_7_1 = data_5_7_0.query('期日.str.contains("1994")', engine='python') #1992のデータを抽出
data_5_7_2 = data_5_7_1.index.values.tolist() #1994/12のデータのindexをlist型に変更
data_5_7 = data_5_7_0[~data_5_7_0.index.isin(data_5_7_2)] #1994/12のデータのindexにあうものを削除
pickle_in.close()
pickle_in_2.close()
pickle_in_3.close()
#display(returns_data_4), display(data_5_11), display(data_5_7)

In [4]:
day_num_6 = data_5_7.期日.nunique(dropna = True) #期日の数(月数)
com_num_6 = data_5_7.shape[0]/day_num_6 #分析可能な企業数
chara_num = data_5_11.shape[1]/int(com_num_6) #用いる企業特性の数
print('分析対象データの月数は',day_num_6,'、', '企業数は',com_num_6,'、', '企業特性数は',chara_num,'である。')

分析対象データの月数は 324 、 企業数は 1141.0 、 企業特性数は 40.0 である。


# 3. 入力データの準備

In [5]:
#右側ファクターネットワークの入力データ
x = returns_data_4.to_numpy()
#左側ベータネットワークの入力データ
y = data_5_11.iloc[:,0:].to_numpy()
x_2 = torch.tensor(x, dtype=torch.float32)
y_2 = torch.tensor(y, dtype=torch.float32)
print('右側ファクターネットワークの入力データの形状は',x.shape,'、', '左側ベータネットワークの入力データの形状は',y.shape,'である。')

右側ファクターネットワークの入力データの形状は (324, 1141) 、 左側ベータネットワークの入力データの形状は (324, 45640) である。


# 4. クラスの生成と学習済みモデルの読み込み

## 4.1 CA$_{1}$アーキテクチャ(左側に第1層を追加、P×1圧縮あり)

In [6]:
class Factors(nn.Module):
    def __init__(self):
        super(Factors, self).__init__()
        
        #層(layer：レイヤー)を定義
        
        ##右側(ファクターネットワーク)の層
        self.fc_r1 = nn.Linear( 
            N*1, #データ(特徴)の入力ユニット数(1059銘柄*1)
            P*1) #出力ユニット数(85企業特性*1)
        
        self.fc_r2 = nn.Linear( 
            P*1, #データ(特徴)の入力ユニット数(85企業特性*1)
            K*1) #出力ユニット数(5ファクター*1)

        ##左側(ベータネットワーク)の層
        self.fc_l1 = nn.Linear(
            N*P, #データ(特徴)の入力ユニット数(1059銘柄*85企業特性)
            lhidden) #出力ユニット数(32)
        
        self.fc_l2 = nn.Linear(
            lhidden, #入力ユニット数(32)
            N*K) #出力結果への出力ユニット数(1059銘柄*5ファクター)
        
    def forward(self, x, y):
        # フォワードパスを定義
        
        #右側(ファクターネットワーク)のフォワードパスを定義
        x = (self.fc_r1(x))
        x = (self.fc_r2(x))
        
        #左側(ベータネットワーク)のフォワードパスを定義
        y = F.relu(self.fc_l1(y)) #ReLU関数を適用
        y = F.relu(self.fc_l2(y)) #ReLU関数を適用
        
        #サイズ自動調整
        
        y = y.view(K, N) #サイズ自動調整
        x = x.view(1, K) #サイズ自動調整
        
        return x, y #返り値

#NNの初期値
N = int(com_num_6) #銘柄数(1059銘柄)
P = int(chara_num)   #企業特性数(85特性)
K = 5    #ファクター数
lhidden = 32  #左側隠れ層1での圧縮次元数

#モデル（Factorsクラス）のインスタンス化
model = Factors()

model = model.to(device)

print(model) #モデルの内容を出力

Factors(
  (fc_r1): Linear(in_features=1141, out_features=40, bias=True)
  (fc_r2): Linear(in_features=40, out_features=5, bias=True)
  (fc_l1): Linear(in_features=45640, out_features=32, bias=True)
  (fc_l2): Linear(in_features=32, out_features=5705, bias=True)
)


## 4.2 学習&バリデーション済みモデルを読み込み

In [7]:
model = Factors()
model = model.to(device)
model.load_state_dict(torch.load('error_rate_min_model'))
model.eval()
#print(list(model.parameters())[0]) #確認用

Factors(
  (fc_r1): Linear(in_features=1141, out_features=40, bias=True)
  (fc_r2): Linear(in_features=40, out_features=5, bias=True)
  (fc_l1): Linear(in_features=45640, out_features=32, bias=True)
  (fc_l2): Linear(in_features=32, out_features=5705, bias=True)
)

# 5. アウトオブサンプルテストの準備

## 5.1 ハイパーパラメーターの読み込みと使用準備

In [8]:
wa_data_0 = pd.read_csv(
    'best_params.txt', sep=" ",header=None)
#print(wa_data_0)
adam_lr_0 = wa_data_0[1].str.strip(',')
alpha_0 = wa_data_0[3].str.strip('}')
adam_lr_1 = pd.DataFrame(adam_lr_0)
alpha_1 = pd.DataFrame(alpha_0)
wa_data_1 = pd.concat([adam_lr_1, alpha_1],axis=1,ignore_index=True)
wa_data_1.columns=['adam_lr', 'alpha']
wa_data_1
adam_lr_2 = float(wa_data_1.iloc[:,0])
alpha_2 = float(wa_data_1.iloc[:,1])
print('学習率は',adam_lr_2,'、','l1アルファは',alpha_2,'がそれぞれ最良の値である。')

学習率は 2.6599310838681845e-05 、 l1アルファは 8.015832747965139e-06 がそれぞれ最良の値である。


## 5.2 損失関数の定義

In [9]:
# 損失関数(specify loss function)は平均二乗誤差
criterion = nn.MSELoss()

# 6. 統計的性能評価

## 6.1 統計的性能評価関数の定義

In [1]:
#OOStestの関数
def OOStest(model, device):
    uvp = pd.DataFrame() #1ヶ月毎の月次収益率の予測値を格納するためのフレーム
    up = pd.DataFrame() #1ヶ月毎のファクターの予測値を格納するためのフレーム
    vp = pd.DataFrame() #1ヶ月毎のベータの予測値を格納するためのフレーム
    OOS_loss = 0
    Numer_t = 0 #total R2の分子
    Numer_p = 0 #pred R2の分子
    Denom = 0 #total R2とpred R2の分母
    t = torch.tensor(0) #時刻
    u = torch.zeros(1,K)
    u_2 = torch.zeros(1,K)
    v = torch.zeros(K,N)
    u = u.to(device)
    u_2 = u_2.to(device)
    v = v.to(device)
    criterion2 = nn.MSELoss(reduction = 'sum') #total R2とpred R2の分子の計算を行うためのMSE
    
    with torch.no_grad():
        for x_input, y_input in zip(xOOS_loader, yOOS_loader):
            t += 1 #いわゆるt
            x_input = x_input.to(device)
            y_input = y_input.to(device)  
            u ,v = model(x_input.float(), y_input.float()) #モデルの出力を取得(左側出力v、右側出力uとする)
            uv = torch.mm(u, v) #最終的に積をとる
            
            loss = criterion(uv, x_input.float()) #入力x.float()と復元outputsの誤差を取得
            OOS_loss += loss.item()  # 誤差(損失)の更新
            
            #pred R2の分子の算出の準備
            u_2 += u
            u_3 = u_2 - u
            
            if t > 1: #tが2以上ならば
                #total R2の分子の算出
                Numer_t += criterion2(x_input.float(), uv).item() #total R2の分子(r_{t} - β_{t-1}f_{t})^2の和
                
                #pred R2の分子の算出
                lamb_t_1 = u_3/(t-1)
                uv_2 = torch.mm(lamb_t_1, v) #積
                Numer_p += criterion2(x_input.float(), uv_2).item() #pred R2の分子の算出のための計算(β_{t-1}λ_{t-1})の和

                #total R2とpred R2の分母の算出
                Denom += torch.sum(torch.square(x_input.float())).item() #total R2とpred R2の分母
            
            #結果(1ヶ月毎の月次収益率の予測値)の格納
            uv2 = uv.to('cpu').detach().numpy().copy() #numpy型に戻す
            uv3 = pd.DataFrame(uv2) #DataFrame型に変換
            uvp = pd.concat([uvp, uv3]) #1ヶ月毎の予測結果の積み上げ格納
            
            #結果(1ヶ月毎のファクターの予測値)の格納
            u2 = u.to('cpu').detach().numpy().copy() #numpy型に戻す
            u3 = pd.DataFrame(u2) #DataFrame型に変換
            up = pd.concat([up, u3]) #1ヶ月毎の予測結果の積み上げ格納
            
            #結果(1ヶ月毎のベータの予測値)の格納
            v2 = v.to('cpu').detach().numpy().copy() #numpy型に戻す
            v3 = pd.DataFrame(v2) #DataFrame型に変換
            vp = pd.concat([vp, v3]) #1ヶ月毎の予測結果の積み上げ格納

                        
        OOS_loss = OOS_loss/len(xOOS_loader) #誤差の算出
        R_total = 1 - (Numer_t/Denom) #total R2の算出
        R_pred = 1 - (Numer_p/Denom) #pred R2の算出
    return OOS_loss, R_total, R_pred, uv, uvp, up, vp

## 6.2 統計的性能評価の実行

In [11]:
num_workers = 1
num_month   = 1
x_batch = num_month 
y_batch = num_month

i_year_3 = 5

xOOS_loader = torch.utils.data.DataLoader(x_2[264:264+(i_year_3*12),:], batch_size=x_batch, num_workers=num_workers,pin_memory=True)
yOOS_loader = torch.utils.data.DataLoader(y_2[264:264+(i_year_3*12),:], batch_size=y_batch, num_workers=num_workers,pin_memory=True)

#OOStest(model, device)
print('誤差は',OOStest(model, device)[0],'、','R_totalは',OOStest(model, device)[1],'、','R_predは',OOStest(model, device)[2],'である。')
#total R^2の結果を保存(特性重要度算出のため)
f = open('R_total.txt', 'w')
f.write(str(OOStest(model, device)[1]))
f.close()

誤差は 0.006564506034677228 、 R_totalは 0.17432298728881113 、 R_predは -0.015696982219724598 である。


# 7. 経済パフォーマンス評価
5区分に分けて(先行研究は10区分)、上位10区分を買い(ロング)、下位10区分を売る(ショート)ような、スプレッドポートフォリオシミュレーションの実行

## 7.1 等加重ポートフォリオ

### 7.1.1 月毎に予測リターンの降順に並べ替え

In [12]:
return_pred_1 = OOStest(model, device)[4]
return_pred_2 = return_pred_1.reset_index()
return_pred_3 = return_pred_2.iloc[:,1:]
return_pred_sorted_4 = pd.DataFrame()
for i in range(return_pred_3.shape[0]): #60ヶ月(アウトオブサンプル期間)
    return_pred_sorted_1 = return_pred_3.iloc[i,:].sort_values(ascending=False,axis=0)
    return_pred_sorted_2 = return_pred_sorted_1.reset_index()
    return_pred_sorted_3 = return_pred_sorted_2.iloc[:,1]
    return_pred_sorted_4 = pd.concat([return_pred_sorted_4, return_pred_sorted_3],axis =1)
display(return_pred_sorted_4)

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.023754,0.036449,0.009212,0.016665,0.062414,0.068667,0.048706,0.007244,0.080669,0.082257,...,0.129433,0.000000,0.014936,0.018398,0.009411,0.043340,0.051082,0.000000,0.000000,0.097525
1,0.022427,0.033340,0.008673,0.014401,0.053003,0.064361,0.046184,0.007141,0.080604,0.077495,...,0.126710,0.000000,0.013199,0.018141,0.008550,0.042527,0.050528,0.000000,0.000000,0.092142
2,0.022048,0.031541,0.008280,0.013866,0.052431,0.063949,0.043707,0.007038,0.079422,0.075735,...,0.123818,0.000000,0.013187,0.018011,0.008103,0.040350,0.050359,0.000000,0.000000,0.091633
3,0.021896,0.031540,0.008249,0.013698,0.049119,0.062487,0.043346,0.007013,0.078264,0.073711,...,0.121933,0.000000,0.013120,0.017497,0.007856,0.039232,0.045888,0.000000,0.000000,0.091381
4,0.021229,0.031043,0.007613,0.013655,0.048120,0.061958,0.042427,0.006966,0.076700,0.073029,...,0.118876,0.000000,0.012886,0.017408,0.007126,0.037172,0.044851,0.000000,0.000000,0.086707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1136,-0.018630,-0.010918,-0.037439,-0.020733,-0.016082,0.000000,-0.009505,-0.023471,0.000000,0.000000,...,0.000000,-0.096745,-0.031411,-0.020737,-0.043884,-0.020836,-0.006343,-0.069926,-0.134337,0.000000
1137,-0.018647,-0.011463,-0.038373,-0.021039,-0.016732,0.000000,-0.009592,-0.023707,0.000000,0.000000,...,0.000000,-0.099265,-0.032140,-0.020949,-0.043898,-0.021232,-0.006620,-0.073674,-0.136415,0.000000
1138,-0.018649,-0.012074,-0.038533,-0.021460,-0.016995,0.000000,-0.009685,-0.023918,0.000000,0.000000,...,0.000000,-0.101720,-0.033055,-0.021425,-0.044635,-0.021848,-0.006651,-0.075058,-0.140561,0.000000
1139,-0.019336,-0.012711,-0.040407,-0.021807,-0.017001,0.000000,-0.010086,-0.024697,0.000000,0.000000,...,0.000000,-0.102026,-0.036366,-0.022326,-0.047433,-0.022059,-0.006832,-0.081254,-0.146616,0.000000


### 7.1.2 月毎に区分毎の平均値を算出

In [13]:
#分位数 = 3
bunnisuu = 353 #(1059/3=353で、P1~P5で一個当たり211企業振り分け)
return_pred_sorted_7 = pd.DataFrame()
for j in range(return_pred_3.shape[0]):
    return_pred_sorted_6 = pd.DataFrame()
    for i in range(int(com_num_6/bunnisuu)):
        return_pred_sorted_5 = return_pred_sorted_4.iloc[bunnisuu*i : bunnisuu*(i+1), j ].mean() 
        return_pred_sorted_6 = pd.concat([return_pred_sorted_6, pd.Series(return_pred_sorted_5)])
    return_pred_sorted_7 = pd.concat([return_pred_sorted_7,return_pred_sorted_6],axis=1)    
display(return_pred_sorted_7)

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0.009516,0.016366,0.001837,0.005803,0.028096,0.037508,0.022340,0.002265,0.045988,0.042752,...,0.070202,-0.009098,0.003456,0.007873,0.000919,0.021305,0.026053,-0.004651,-0.016432,0.052901
0,0.001154,0.006642,-0.004135,-0.000328,0.013156,0.020933,0.009594,-0.001609,0.024811,0.023661,...,0.039701,-0.026074,-0.003488,0.000657,-0.006985,0.008342,0.012802,-0.016831,-0.042521,0.029062
0,-0.005439,-0.000059,-0.015318,-0.007509,0.001974,0.008548,0.000537,-0.008910,0.009582,0.009119,...,0.016148,-0.046555,-0.013382,-0.006270,-0.019471,-0.001132,0.003667,-0.034710,-0.073337,0.011667


### 7.1.3 評価指標を算出

In [14]:
longshort_all = return_pred_sorted_7.max() - return_pred_sorted_7.min() #ロング-ショート(P1 - P5)、なお月次であることに注意
longshort_return = longshort_all.mean()
longshort_std = longshort_all.std()
longshort_sharperatio_month = longshort_return/longshort_std
print('期待収益率は',longshort_return,'、','標準偏差は',longshort_std,'、','(月次)シャープレシオは',longshort_sharperatio_month,'である。')

期待収益率は 0.033423136334810505 、 標準偏差は 0.020461548594769893 、 (月次)シャープレシオは 1.6334607412536546 である。


## 7.2 バリューウエイトポートフォリオ

### 7.2.1 時価総額データの読み込み
基準となる時価総額の値を2017/01に設定

In [15]:
pickle_in = open("./organized_data3.pickle","rb")
data_4_6 = pickle.load(pickle_in)
pickle_in.close()
value_data_1 = data_4_6.query('期日.str.contains("2017/01")', engine='python') #2017/01のデータを抽出
value_data_2 = value_data_1.reset_index()
value_data_3 = value_data_2['時価総額(発行済株式数ﾍﾞｰｽ)']
display(value_data_3)

0       2.974602e+10
1       1.737111e+11
2       1.097649e+10
3       1.743433e+10
4       1.566064e+11
            ...     
1136    3.898652e+10
1137    2.016640e+10
1138    1.860260e+10
1139    1.503639e+10
1140    8.771760e+10
Name: 時価総額(発行済株式数ﾍﾞｰｽ), Length: 1141, dtype: float64

### 7.2.2 月毎に区分毎の平均値を算出

In [16]:
return_pred_1 = OOStest(model, device)[4]
#分位数 = 3
bunnisuu = 353 #(1059/3=353で、P1~P5で一個当たり211企業振り分け)
value_data_9 = pd.DataFrame()
for q in range(60):
    value_data_8 = pd.DataFrame()
    value_data_4 = pd.concat([return_pred_1.T.iloc[:,q],value_data_3], axis=1) #予測リターンと時価総額を結合
    value_data_5 = value_data_4.sort_values(0, ascending=False) #予測リターンの値で降順にソート
    value_data_6 = value_data_5.reset_index().iloc[:,1:]
    for p in range(int(com_num_6/bunnisuu)):
        value_data_7 = sum(value_data_6.iloc[bunnisuu*p : bunnisuu*(p+1), 0] * value_data_6.iloc[bunnisuu*p : bunnisuu*(p+1), 1])/sum(value_data_6.iloc[bunnisuu*p : bunnisuu*(p+1), 1])
        value_data_8 = pd.concat([value_data_8, pd.Series(value_data_7)])
    value_data_9 = pd.concat([value_data_9,value_data_8],axis=1) 
display(value_data_9)

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0.009454,0.016103,0.002029,0.005699,0.027869,0.037825,0.022804,0.002071,0.045611,0.042866,...,0.068373,-0.008515,0.003400,0.008024,0.000889,0.022059,0.025751,-0.004770,-0.017004,0.050275
0,0.000694,0.006304,-0.003814,-0.000484,0.013431,0.020252,0.009396,-0.001602,0.023804,0.022451,...,0.038981,-0.026834,-0.003082,0.000595,-0.007220,0.008720,0.012223,-0.016165,-0.042797,0.027772
0,-0.004901,-0.000392,-0.013328,-0.006695,0.001527,0.009065,0.000828,-0.009294,0.009873,0.008660,...,0.015817,-0.045094,-0.012351,-0.007114,-0.017934,-0.001390,0.003694,-0.032344,-0.075089,0.011588


### 7.2.3 評価指標を算出

In [17]:
longshort2_all = value_data_9.max() - value_data_9.min() #ロング-ショート(P1 - P5)、なお月次であることに注意
longshort2_return = longshort2_all.mean()
longshort2_std = longshort2_all.std()
longshort2_sharperatio_month = longshort2_return/longshort2_std
print('期待収益率は',longshort2_return,'、','標準偏差は',longshort2_std,'、','(月次)シャープレシオは',longshort2_sharperatio_month,'である。')

期待収益率は 0.032765997952967564 、 標準偏差は 0.019970394908797363 、 (月次)シャープレシオは 1.6407285936310394 である。


## 7.3 タンジェンシー(接点)ポートフォリオ

### $β_{t} : K × N,　　Σ_{t} : K × K,　　W_{t} : N × 1,　　μ : N × 1$
$V[R_{p}] = W_{t}'Σ W_{t} = W_{t}'(β_{t}' Σ_{t} β_{t})W_{t},　 E[R_{p}] = W_{t}'μ$

$E[R_{p}] - λV[R_{p}] = Σ_{i=1}^{N} W_{i}^2$

max$E[R_{p}]$で$V[R_{p}]=1$の条件付き

TOPIXで月率5とか

### 7.3.1 時点$t$毎にファクターの予測値(右側ファクターネットワークの出力)をデータフレーム化して表示

In [18]:
factor_pred_1 = OOStest(model, device)[5]
factor_pred_2 = factor_pred_1.reset_index()
factor_pred_3 = factor_pred_2.iloc[:,1:]
factor_pred_3.columns =  ["F1","F2","F3","F4","F5"] #列名の変更
display(factor_pred_3) #最初の10ヶ月のみ表示

,F1,F2,F3,F4,F5
0,-0.011995,0.070395,-0.025991,-0.092467,0.067965
1,0.007860,0.094317,0.026725,-0.066029,0.076092
2,-0.042975,0.018362,-0.029330,-0.149631,0.033495
3,-0.000061,0.046118,-0.029792,-0.102876,0.044308
4,0.033341,0.119814,0.099617,-0.100357,0.129457
5,0.067477,0.166805,0.095903,0.011096,0.122027
6,-0.015466,0.097051,0.042368,-0.043247,0.117181
7,-0.026516,0.020319,-0.003690,-0.098313,0.020978
8,0.042784,0.190027,0.123027,0.023297,0.162459
9,0.037969,0.155293,0.113333,0.029940,0.165013


### 7.3.2 $t-1$までのファクター予測値を用いて時点$t$の平均値を算出

In [19]:
factor_pred_5 = pd.DataFrame()
for f in range(1, 61):
    factor_pred_4 = pd.DataFrame(factor_pred_3.iloc[0:f,:].mean())
    factor_pred_5 = pd.concat([factor_pred_5, factor_pred_4], axis=1) #予測リターンと時価総額を結合
factor_pred_6 = factor_pred_5.T.reset_index().iloc[:,1:]
display(factor_pred_6.head(10))

,F1,F2,F3,F4,F5
0,-0.011995,0.070395,-0.025991,-0.092467,0.067965
1,-0.002068,0.082356,0.000367,-0.079248,0.072028
2,-0.015703,0.061024,-0.009532,-0.102709,0.059184
3,-0.011793,0.057298,-0.014597,-0.102751,0.055465
4,-0.002766,0.069801,0.008246,-0.102272,0.070263
5,0.008941,0.085968,0.022855,-0.083377,0.078890
6,0.005454,0.087552,0.025643,-0.077644,0.084361
7,0.001458,0.079147,0.021976,-0.080228,0.076438
8,0.006050,0.091467,0.033204,-0.068725,0.085996
9,0.009242,0.097850,0.041217,-0.058859,0.093897


### 7.3.3 時点$t$における分散共分散(共分散)行列を求める

In [20]:
from math import sqrt, ceil
eps = 1e-10
def find_size(length):
    return ceil(sqrt(1+8*length)/2 - 0.5 - eps)
cov_dataframe = pd.DataFrame()
for t in range(1,61):
#for t in range(60):
    factor_pred_7 = pd.DataFrame(factor_pred_3.iloc[0:t+1,:])
    cov_matrix_1 = np.empty(0)
    for i in range(factor_pred_7.shape[1]):
        for j in range(factor_pred_7.shape[1]):
            factor_pred_8_1 = factor_pred_7.iloc[:,[i,j]]
            factor_pred_8_2, factor_pred_8_3 = factor_pred_8_1.iloc[:,0],  factor_pred_8_1.iloc[:,1]
            #print(factor_pred_8_2, factor_pred_8_3)
            if i <= j:
                #print(i+1, j+1)
                cov = np.cov(np.array(factor_pred_8_2), np.array(factor_pred_8_3), bias=1)
                cov_ij = cov[0,1]
                #print(cov_ij)
                cov_matrix_1 = np.hstack([cov_matrix_1, cov_ij])
    #print(cov_matrix_1)
    vv = cov_matrix_1
    size = find_size(len(vv))
    tm = np.zeros(size * size, dtype=vv.dtype).reshape(size, size)
    position = [0] + list(np.arange(size, 0,  -1).cumsum())
    for i in range(size):
        tm[i:,i] = vv[position[i]:position[i+1]]
    diag = np.diag(tm)
    diag_2 = np.diag(diag)
    cov_matrix_2 = tm+tm.T-diag_2
    #print("t=", t)
    #print(cov_matrix_2)
    cov_dataframe = pd.concat([cov_dataframe, pd.DataFrame(cov_matrix_2)], axis=0)
cov_dataframe.columns =  ["F1","F2","F3","F4","F5"] #列名の変更
cov_dataframe_1 = cov_dataframe.reset_index()
cov_dataframe_2 = cov_dataframe_1.iloc[:,1:]
display(cov_dataframe_2)

,F1,F2,F3,F4,F5
0,0.000099,0.000119,0.000262,0.000131,0.000040
1,0.000119,0.000143,0.000315,0.000158,0.000049
2,0.000262,0.000315,0.000695,0.000348,0.000107
3,0.000131,0.000158,0.000348,0.000175,0.000054
4,0.000040,0.000049,0.000107,0.000054,0.000017
...,...,...,...,...,...
295,0.005908,0.009320,0.012884,0.009162,0.009514
296,0.009320,0.016146,0.022316,0.016055,0.016489
297,0.012884,0.022316,0.032055,0.022698,0.023508
298,0.009162,0.016055,0.022698,0.016993,0.016797


### 7.3.4 時点$t$における$β_{t}' Σ_{t} β_{t}$を求める

In [21]:
beta_pred_1 = OOStest(model, device)[6]
dot_dataframe = pd.DataFrame()
for d in range(60):
    cov_dataframe_3 = cov_dataframe_2.iloc[5*d:5*(d+1),:] #K×K行列
    beta_pred_2 = beta_pred_1.iloc[5*d:5*(d+1),:] #K×N行列
    cov_dataframe_4 = np.array(cov_dataframe_3)
    beta_pred_3 = np.array(beta_pred_2)
    dot_1 = np.dot(cov_dataframe_4, beta_pred_3)
    dot_2 = np.dot(beta_pred_3.T, dot_1) #N×N行列
    dot_dataframe = pd.concat([dot_dataframe, pd.DataFrame(dot_2)], axis=0)
dot_dataframe_1 = dot_dataframe.reset_index()
dot_dataframe_2 = dot_dataframe_1.iloc[:,1:]
display(dot_dataframe_2)

,0,1,2,3,4,5,6,7,8,9,...,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140
0,0.000014,0.000025,0.000020,0.000006,0.000006,0.000007,0.000013,0.000008,0.000024,0.000004,...,0.000020,1.921913e-06,0.000020,0.000010,0.000014,0.000016,0.000003,0.0,3.713903e-07,0.000014
1,0.000025,0.000045,0.000036,0.000010,0.000011,0.000013,0.000022,0.000015,0.000044,0.000008,...,0.000036,3.437894e-06,0.000036,0.000018,0.000026,0.000029,0.000006,0.0,6.643382e-07,0.000024
2,0.000020,0.000036,0.000028,0.000008,0.000009,0.000010,0.000018,0.000012,0.000035,0.000006,...,0.000028,2.721629e-06,0.000029,0.000014,0.000020,0.000023,0.000005,0.0,5.259273e-07,0.000019
3,0.000006,0.000010,0.000008,0.000002,0.000002,0.000003,0.000005,0.000003,0.000010,0.000002,...,0.000008,7.589948e-07,0.000008,0.000004,0.000006,0.000006,0.000001,0.0,1.466681e-07,0.000005
4,0.000006,0.000011,0.000009,0.000002,0.000003,0.000003,0.000006,0.000004,0.000011,0.000002,...,0.000009,8.469810e-07,0.000009,0.000004,0.000006,0.000007,0.000002,0.0,1.636705e-07,0.000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68455,0.001834,0.001466,0.001676,0.000199,0.001500,0.000207,0.001111,0.000457,0.001416,0.000413,...,0.001297,1.493222e-04,0.001166,0.000721,0.000561,0.000675,0.000595,0.0,9.383682e-05,0.000889
68456,0.001620,0.001314,0.001484,0.000176,0.001352,0.000183,0.000987,0.000409,0.001254,0.000374,...,0.001148,1.320090e-04,0.001040,0.000645,0.000494,0.000595,0.000542,0.0,8.351554e-05,0.000783
68457,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000
68458,0.000255,0.000206,0.000231,0.000027,0.000213,0.000029,0.000158,0.000068,0.000195,0.000058,...,0.000185,2.067142e-05,0.000160,0.000100,0.000079,0.000094,0.000084,0.0,1.442489e-05,0.000127


### 7.3.5 時点$t$における予測リターンをデータフレーム化

In [22]:
return_pred_1 = OOStest(model, device)[4] #1ヶ月毎の予測リターン
return_pred_2 = return_pred_1.reset_index()
return_pred_3 = return_pred_2.iloc[:,1:]
display(return_pred_3.head(10))

,0,1,2,3,4,5,6,7,8,9,...,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140
0,0.002373,0.009470,-0.009078,-0.010390,0.009529,0.005040,-0.000855,-0.001980,-0.015393,0.008157,...,0.006867,0.003861,-0.012003,0.012536,0.004149,0.015806,0.011122,0.0,-0.000119,0.011956
1,0.008945,0.021904,0.001204,-0.007434,0.014570,0.008760,0.008186,0.003600,-0.002251,0.010208,...,0.015362,0.004210,-0.000256,0.019165,0.012835,0.028857,0.013672,0.0,0.000167,0.020925
2,-0.019245,-0.013888,-0.011235,-0.019058,0.000227,-0.000270,-0.012771,-0.005716,-0.026741,0.001555,...,-0.008559,-0.000035,-0.019855,-0.009123,-0.001937,0.002379,0.005351,0.0,-0.002544,-0.004263
3,-0.007278,0.004138,-0.009987,-0.010408,0.006547,0.002021,-0.004100,-0.001022,-0.019284,0.005132,...,0.004781,0.000296,-0.015875,0.002883,0.002233,0.006838,0.008062,0.0,0.000479,0.009116
4,0.006547,0.046435,0.008124,-0.010831,0.024210,0.011384,0.013749,0.007793,0.004348,0.020146,...,0.031093,0.005820,0.006656,0.022215,0.017128,0.035870,0.023790,0.0,0.000527,0.028795
5,0.035112,0.054671,0.017971,0.001270,0.023611,0.018992,0.018158,0.017388,0.018733,0.025889,...,0.045628,0.011836,0.019501,0.037622,0.026936,0.040169,0.023632,0.0,0.000000,0.040297
6,0.017378,0.024346,0.003499,-0.004877,0.021667,0.005905,0.003446,0.010278,0.001702,0.022060,...,0.015551,0.013499,0.009797,0.032301,0.016487,0.026369,0.022059,0.0,0.000631,0.017895
7,-0.008075,-0.005857,-0.007643,-0.011447,0.001445,0.001000,-0.005946,-0.003487,-0.013603,0.001464,...,-0.002460,0.001818,-0.007633,-0.000774,0.001599,0.003990,0.002302,0.0,-0.000590,-0.000480
8,0.037686,0.067198,0.020987,0.002757,0.037603,0.015458,0.019405,0.020172,0.020843,0.039431,...,0.048291,0.014373,0.025455,0.055059,0.026155,0.055759,0.032284,0.0,0.000257,0.046427
9,0.039828,0.054138,0.019782,0.003970,0.033928,0.016804,0.019211,0.022043,0.018110,0.034979,...,0.041755,0.019469,0.028710,0.051728,0.032441,0.040179,0.033270,0.0,0.000820,0.035305


In [23]:
import cvxopt as opt
dot_dataframe_2, return_pred_3
P = opt.matrix((-2)*np.array(dot_dataframe_2.iloc[0:1059,:])) #N×Nのcovariance matrix(cvxoptの関係で×(-2)を施している)
Q_0 = np.array(-return_pred_3.iloc[0,:]) #(cvxoptの関係で×(-1)を施している)
Q_0 = Q_0.astype(np.double)
Q = opt.matrix(Q_0)

# constraints Ax = b
A = opt.matrix(np.ones(return_pred_3.shape[1])).T
b = opt.matrix(1.0) # sum(w) = 1

#G = opt.matrix(np.zeros((1059, 1059, )))
#h = opt.matrix(np.zeros(1059))
G = opt.matrix(np.ones((1059, 1059, )))
h = opt.matrix(np.ones(1059))

# convex optimization
opt.solvers.options['show_progress'] = False
sol = opt.solvers.qp(P, Q, G, h, A, b)

TypeError: 'P' must be a 'd' matrix of size (1141, 1141)

In [ ]:
P, Q, A

In [ ]:
a = np.zeros((1059, 1059, ))
a.shape

In [ ]:
np.ones(1059).shape

In [ ]:
P, Q

In [ ]:
import scipy.optimize as sco



def max_func_var(weights, dot_dataframe_3, return_pred_4):
    return - np.dot(return_pred_4.T, weights) + np.dot(weights.T, np.dot(dot_dataframe_3, weights))

return_pred_4 = np.array(return_pred_3.iloc[0,:])
dot_dataframe_3 = np.array(dot_dataframe_2.iloc[0:1059,:])

x0 = [1. / 1059] * 1059

tret = 0  # e.g.: tret = 0.5

cons = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]

bnds = [(0, None)] * 1059

opts = sco.minimize(fun=max_func_var, x0=x0, method='SLSQP', bounds=bnds, constraints=cons)

In [ ]:


import cvxopt as opt
dot_dataframe_2, return_pred_3
P = opt.matrix((-2)*np.array(dot_dataframe_2.iloc[0:1059,:])) #N×Nのcovariance matrix(cvxoptの関係で×(-2)を施している)
Q_0 = np.array(-return_pred_3.iloc[0,:]) #(cvxoptの関係で×(-1)を施している)
Q_0 = Q_0.astype(np.double)
Q = opt.matrix(Q_0)

# constraints Ax = b
A = opt.matrix(np.ones(return_pred_3.shape[1])).T
b = opt.matrix(1.0) # sum(w) = 1

#G = opt.matrix(np.zeros((1059, 1059, )))
#h = opt.matrix(np.zeros(1059))
G = opt.matrix(np.ones((1059, 1059, )))
h = opt.matrix(np.ones(1059))

# convex optimization
opt.solvers.options['show_progress'] = False
sol = opt.solvers.qp(P, Q, G, h, A, b)

# 分散共分散確認用コード
np.var(np.array(factor_pred_7)), np.cov(np.array(factor_pred_7))
np.cov(np.array(factor_pred_7.iloc[:,[0,1]]))
np.var(factor_pred_7["F1"]), np.var(factor_pred_7["F2"]), np.cov(factor_pred_7)


np.cov(np.array([factor_pred_7["F1"]]), np.array([factor_pred_7["F1"]]), bias=1), np.cov(np.array([factor_pred_7["F1"]]), np.array([factor_pred_7["F2"]]), bias=1), np.cov(np.array(factor_pred_7["F2"]), bias=1), np.var(np.array([factor_pred_7["F2"]]))